In [ ]:
from typing import Literal
import pandas as pd
import ast
import matplotlib.pyplot as plt
import os
import math
import re
import csv
import io


# Read in data
master_df = pd.read_csv(
    "data/synthetic/public/standard_nd_aggregate_responses.csv",
    encoding="utf-8",
    encoding_errors="ignore",
    on_bad_lines="skip",
)

make_all_graphs(master_df)
